# Autism Detection

In [93]:
# Importing the libraries

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, StandardScaler, OneHotEncoder
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score, KFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.compose import ColumnTransformer
from pprint import pprint
from sklearn.model_selection import train_test_split
#from keras.utils import to_categorical

**EDA**

In [94]:
# STEP - 0
# # Import the dataset
from google.colab import files
uploaded = files.upload()


Saving Phenotypic_V1_0b_preprocessed1.csv to Phenotypic_V1_0b_preprocessed1 (5).csv


In [111]:
import io
df_pheno = pd.read_csv(io.BytesIO(uploaded['Phenotypic_V1_0b_preprocessed1.csv']))
# Dataset is now stored in a Pandas Dataframe

#df_pheno = pd.read_csv('Phenotypic_V1_0b_preprocessed1.csv', encoding='unicode_escape')
display(df_pheno.head(n=5))
#Total Coulmns
columns_pheno = df_pheno.columns
print("columns", columns_pheno.size)
print("total", df_pheno.size)

# Total number of records
n_records = len(df_pheno.index)

# Number of records where individual's with ASD
n_asd_yes = len(df_pheno[df_pheno['DX_GROUP'] == 1])

# Number of records where individual's with no ASD
n_asd_no = len(df_pheno[df_pheno['DX_GROUP'] == 2])

# Percentage of individuals whose are with ASD
yes_percent = float(n_asd_yes) / n_records *100



# Print the results
print("Total number of records:", n_records)
print ("Individuals diagonised with ASD: {}".format(n_asd_yes))
print ("Individuals not diagonised with ASD: {}".format(n_asd_no))
print ("Percentage of individuals diagonised with ASD: {:.2f}%".format(yes_percent))

df_pheno.describe()



,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,...,WISC_IV_SIM_SCALED,WISC_IV_VOCAB_SCALED,WISC_IV_INFO_SCALED,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr,func_efc,func_fber,func_fwhm,func_dvars,func_outlier,func_quality,func_mean_fd,func_num_fd,func_perc_fd,func_gsr,qc_rater_1,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1.0,16.77,1,Ambi,NaN,103.0,116.0,89.0,WASI,WASI,WASI,16.0,9.0,5.0,4.0,1.0,4.0,12.0,4.0,8.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,10.201539,1.194664,16.223458,3.878000,0.152711,12.072452,0.613128,45.446551,1.873339,1.054931,0.000641,0.011443,0.116828,8.0,3.980100,0.054346,fail,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1.0,24.45,1,R,NaN,124.0,128.0,115.0,WASI,WASI,WASI,27.0,22.0,5.0,3.0,1.0,4.0,13.0,5.0,8.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,7.165701,1.126752,10.460008,4.282238,0.161716,9.241155,0.578301,56.286350,2.012112,0.949857,0.000474,0.031781,0.322092,135.0,67.164179,0.041862,OK,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1.0,19.09,1,R,NaN,113.0,108.0,117.0,WASI,WASI,WASI,19.0,12.0,5.0,3.0,1.0,4.0,18.0,6.0,12.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,7.698144,1.226218,9.725750,3.881684,0.174186,9.323463,0.578960,63.317943,1.866104,1.180605,0.008262,0.014260,0.127745,29.0,14.427861,0.046745,OK,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1.0,13.73,2,R,NaN,119.0,117.0,118.0,WASI,WASI,WASI,23.0,19.0,3.0,4.0,1.0,4.0,12.0,4.0,8.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,9.071807,1.256278,11.198226,3.628667,0.119269,10.814200,0.556064,70.800354,1.918278,1.092030,0.001711,0.019205,0.128136,22.0,10.945274,0.027963,OK,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1.0,13.37,1,L,NaN,109.0,99.0,119.0,WASI,WASI,WASI,13.0,10.0,4.0,3.0,1.0,4.0,12.0,4.0,8.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,8.026798,1.407166,6.282055,3.674539,0.130647,10.123574,0.562942,75.364679,2.213873,1.086830,0.001500,0.006919,0.070143,3.0,1.492537,0.054006,OK,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


columns 106
total 117872
Total number of records: 1112
Individuals diagonised with ASD: 539
Individuals not diagonised with ASD: 573
Percentage of individuals diagonised with ASD: 48.47%


,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,SCQ_TOTAL,AQ_TOTAL,CURRENT_MED_STATUS,OFF_STIMULANTS_AT_SCAN,VINELAND_RECEPTIVE_V_SCALED,...,VINELAND_PLAY_V_SCALED,VINELAND_COPING_V_SCALED,VINELAND_SOCIAL_STANDARD,VINELAND_SUM_SCORES,VINELAND_ABC_STANDARD,VINELAND_INFORMANT,WISC_IV_VCI,WISC_IV_PRI,WISC_IV_WMI,WISC_IV_PSI,WISC_IV_SIM_SCALED,WISC_IV_VOCAB_SCALED,WISC_IV_INFO_SCALED,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr,func_efc,func_fber,func_fwhm,func_dvars,func_outlier,func_quality,func_mean_fd,func_num_fd,func_perc_fd,func_gsr,SUB_IN_SMP
count,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1112.000000,1040.000000,1112.000000,1112.000000,364.000000,1040.000000,917.000000,931.000000,378.000000,379.000000,378.000000,297.000000,391.000000,507.000000,415.000000,390.000000,391.000000,331.000000,376.000000,265.000000,270.000000,273.000000,273.000000,231.000000,365.000000,64.000000,64.000000,64.000000,64.00000,64.000000,124.000000,58.000000,809.000000,72.000000,112.000000,...,112.000000,112.000000,112.000000,112.000000,112.000000,112.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,55.000000,1112.000000,100.000000,94.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1099.000000,1112.000000
mean,555.500000,556.500000,50752.684353,556.500000,50752.684353,1.515288,0.639423,17.048864,1.147482,60.567225,108.380962,107.812432,106.625134,19.767196,15.791557,6.084656,3.218855,0.910486,3.341223,11.062651,3.553846,7.539642,1.921450,0.928191,9.071698,2.840741,11.912088,6.758242,1.199134,57.882192,7.546875,10.000000,17.406250,10.37500,8.937500,11.596774,21.155172,0.168109,0.333333,12.883929,...,12.616071,14.794643,89.089286,272.035714,89.187500,1.035714,112.109091,106.618182,100.690909,96.690909,12.381818,11.818182,12.127273,11.636364,9.909091,11.545455,10.127273,10.363636,8.763636,9.890909,1.312050,12.951700,20.519255,11.641527,2.079628,72.804971,3.558793,0.072209,48.185793,0.508707,100.548498,2.116388,1.103531,0.005525,0.012778,0.131138,27.218380,12.952826,0.028416,0.686151
std,321.151055,321.151055,447.680666,321.151055,447.680666,0.499991,0.828849,8.036419,0.354745,47.495811,15.072062,16.244305,15.339533,5.527245,4.633822,2.577855,1.261132,0.285850,0.498948,4.638208,1.750776,3.337870,1.619912,0.258515,4.223688,1.857519,5.321948,2.486762,0.400216,42.799043,4.120909,7.690439,12.852915,7.54142,7.028005,9.602608,11.575536,0.374194,0.474713,3.256997,...,4.020769,3.785078,23.033291,58.131093,19.839233,0.186411,17.142834,13.967543,15.143600,14.309894,3.094146,3.646870,3.103490,3.014555,3.187201,2.651472,3.232099,2.914610,2.821634,2.753266,0.463539,2.193555,5.244446,5.612932,11.435278,155.033640,0.676312,0.052283,282.736653,0.059832,43.552970,0.345282,0.117803,0.006464,0.011212,0.157728,42.757583,18.216671,0.021070,0.464265
min,0.000000,1.000000,50002.000000,1.000000,50002.000000,1.000000,0.000000,6.470000,1.000000,-100.000000,41.000000,42.000000,37.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,7.000000,

In [96]:
#Total Males
males = df_pheno[df_pheno['SEX'] == 1]
print("males:", len(males))
## Autistic Males
males_autistic = males[males['DX_GROUP'] == 1]
print("males_autistic:", len(males_autistic))
## Control Males
males_control = males[males['DX_GROUP'] == 2]
print("males_control:", len(males_control))

#Total Females
females = df_pheno[df_pheno['SEX'] == 2]
print("females:", len(females))
## Autistic Females
females_autistic = females[females['DX_GROUP'] == 1]
print("females_autistic:", len(females_autistic))
## Control Females
females_controls = females[females['DX_GROUP'] == 2]
print("females_controls:", len(females_controls))

# # Creating dataset with selected features only
# pheno_data = dataset.loc[:,['SUB_ID','SEX','HANDEDNESS_CATEGORY','AGE_AT_SCAN','FIQ','VIQ','PIQ','ADOS_TOTAL',
#                             'ADI_R_SOCIAL_TOTAL_A','ADI_R_VERBAL_TOTAL_BV','ADI_RRB_TOTAL_C','ADI_R_ONSET_TOTAL_D',
#                             'SUB_IN_SMP','CURRENT_MED_STATUS','DSM_IV_TR']]


males: 948
males_autistic: 474
males_control: 474
females: 164
females_autistic: 65
females_controls: 99


** Filtering **

In [97]:
#Filter Data Set
#selection = filter_data[(filter_data['AGE_AT_SCAN '] <= 40) & (filter_data['FIQ'] >= 80)]    # add check for FIQ = NaN
selection = df_pheno[(df_pheno['AGE_AT_SCAN'] <= 40) & (df_pheno['FIQ'] >= 80)]    # add check for FIQ = NaN
selection = selection.dropna(subset=['FIQ','DX_GROUP'], axis=0) # drop the row if FIQ or DX_GROUP is null

# columns_sel = selection.columns
# print("columns", columns_sel.size) # 2 null FIQ (351-349)
# selection = selection[['SEX','DX_GROUP','SUB_ID']]
## Total Females-Males
females = selection[selection['SEX'] == 2]
males = selection[selection['SEX'] == 1]
print("females", len(females))
print("males", len(males))
## Autistic Females-Males
females_autistic = females[females['DX_GROUP'] == 1]
males_autistic = males[males['DX_GROUP'] == 1]
males_control = males[males['DX_GROUP'] == 2]
print("females_autistic", len(females_autistic))
print("males_autistic", len(males_autistic))
print("males_control", len(males_control))
# # therefore we need len(females_autistic) control males and len(females)-len(females_autistic) autistic males
# # remove: #males-#autistic_females and #males
# ma_to_remove = np.random.choice(males_autistic.index, size=240, replace=False)
# mc_to_remove = np.random.choice(males_control.index, size=330, replace=False)
# selection = selection.drop(ma_to_remove)
# selection = selection.drop(mc_to_remove)
# labels = selection[['SUB_ID','DX_GROUP']]
# labels_dict = labels.set_index('SUB_ID').T.to_dict('list')
records = len(selection.index)
print("Total:",records)
select_yes = len(selection[selection['DX_GROUP'] == 1])
print("ASD:",select_yes)
y_percent = float(select_yes) / n_records *100
print("ASD percent",y_percent)
display(selection.head(n=5))

females 149
males 833
females_autistic 57
males_autistic 401
males_control 432
Total: 982
ASD: 458
ASD percent 41.18705035971223


,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,FIQ_TEST_TYPE,VIQ_TEST_TYPE,PIQ_TEST_TYPE,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,...,WISC_IV_SIM_SCALED,WISC_IV_VOCAB_SCALED,WISC_IV_INFO_SCALED,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr,func_efc,func_fber,func_fwhm,func_dvars,func_outlier,func_quality,func_mean_fd,func_num_fd,func_perc_fd,func_gsr,qc_rater_1,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,0,1,50002,1,50002,PITT,no_filename,1,1.0,16.77,1,Ambi,NaN,103.0,116.0,89.0,WASI,WASI,WASI,16.0,9.0,5.0,4.0,1.0,4.0,12.0,4.0,8.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,10.201539,1.194664,16.223458,3.878000,0.152711,12.072452,0.613128,45.446551,1.873339,1.054931,0.000641,0.011443,0.116828,8.0,3.980100,0.054346,fail,NaN,OK,NaN,fail,ic-parietal-cerebellum,OK,NaN,fail,ERROR #24,1
1,1,2,50003,2,50003,PITT,Pitt_0050003,1,1.0,24.45,1,R,NaN,124.0,128.0,115.0,WASI,WASI,WASI,27.0,22.0,5.0,3.0,1.0,4.0,13.0,5.0,8.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,7.165701,1.126752,10.460008,4.282238,0.161716,9.241155,0.578301,56.286350,2.012112,0.949857,0.000474,0.031781,0.322092,135.0,67.164179,0.041862,OK,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,2,3,50004,3,50004,PITT,Pitt_0050004,1,1.0,19.09,1,R,NaN,113.0,108.0,117.0,WASI,WASI,WASI,19.0,12.0,5.0,3.0,1.0,4.0,18.0,6.0,12.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,7.698144,1.226218,9.725750,3.881684,0.174186,9.323463,0.578960,63.317943,1.866104,1.180605,0.008262,0.014260,0.127745,29.0,14.427861,0.046745,OK,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
3,3,4,50005,4,50005,PITT,Pitt_0050005,1,1.0,13.73,2,R,NaN,119.0,117.0,118.0,WASI,WASI,WASI,23.0,19.0,3.0,4.0,1.0,4.0,12.0,4.0,8.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,9.071807,1.256278,11.198226,3.628667,0.119269,10.814200,0.556064,70.800354,1.918278,1.092030,0.001711,0.019205,0.128136,22.0,10.945274,0.027963,OK,NaN,OK,NaN,maybe,ic-parietal-cerebellum,OK,NaN,OK,NaN,0
4,4,5,50006,5,50006,PITT,Pitt_0050006,1,1.0,13.37,1,L,NaN,109.0,99.0,119.0,WASI,WASI,WASI,13.0,10.0,4.0,3.0,1.0,4.0,12.0,4.0,8.0,4.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,8.026798,1.407166,6.282055,3.674539,0.130647,10.123574,0.562942,75.364679,2.213873,1.086830,0.001500,0.006919,0.070143,3.0,1.492537,0.054006,OK,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


In [98]:
#corelation between the orignal data
correlation_o = df_pheno.corr()
correlation_o

,Unnamed: 0,Unnamed: 0.1,SUB_ID,X,subject,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,SCQ_TOTAL,AQ_TOTAL,CURRENT_MED_STATUS,OFF_STIMULANTS_AT_SCAN,VINELAND_RECEPTIVE_V_SCALED,...,VINELAND_PLAY_V_SCALED,VINELAND_COPING_V_SCALED,VINELAND_SOCIAL_STANDARD,VINELAND_SUM_SCORES,VINELAND_ABC_STANDARD,VINELAND_INFORMANT,WISC_IV_VCI,WISC_IV_PRI,WISC_IV_WMI,WISC_IV_PSI,WISC_IV_SIM_SCALED,WISC_IV_VOCAB_SCALED,WISC_IV_INFO_SCALED,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr,func_efc,func_fber,func_fwhm,func_dvars,func_outlier,func_quality,func_mean_fd,func_num_fd,func_perc_fd,func_gsr,SUB_IN_SMP
Unnamed: 0,1.000000,1.000000,0.996802,1.000000,0.996802,0.061703,0.047760,0.181828,0.014324,-0.054216,-0.018130,-0.014932,0.016984,-0.031584,-0.027844,-0.055876,-0.039102,-0.062540,-0.105054,0.046482,-0.019601,0.067449,0.045814,-0.038527,0.121982,0.009640,0.099288,0.128881,-0.188254,-0.072971,0.138917,0.147769,0.128280,0.159167,0.156117,-0.258679,0.241644,-0.052009,0.748119,0.515331,...,0.527677,0.637955,0.630992,0.574395,0.591557,-0.144162,-0.121505,0.036496,-0.115563,-0.124800,-0.072100,-0.126149,-0.107639,0.205908,-0.095745,-0.019618,-0.040055,-0.170137,-0.051207,-0.153658,0.181235,-0.134917,0.254167,-0.117261,-0.237504,0.123073,-0.179085,-0.169309,0.163762,0.173439,-0.049783,-0.185545,-0.022723,-0.037161,-0.120263,-0.060259,-0.130511,-0.047600,-0.096649,-0.152313
Unnamed: 0.1,1.000000,1.000000,0.996802,1.000000,0.996802,0.061703,0.047760,0.181828,0.014324,-0.054216,-0.018130,-0.014932,0.016984,-0.031584,-0.027844,-0.055876,-0.039102,-0.062540,-0.105054,0.046482,-0.019601,0.067449,0.045814,-0.038527,0.121982,0.009640,0.099288,0.128881,-0.188254,-0.072971,0.138917,0.147769,0.128280,0.159167,0.156117,-0.258679,0.241644,-0.052009,0.748119,0.515331,...,0.527677,0.637955,0.630992,0.574395,0.591557,-0.144162,-0.121505,0.036496,-0.115563,-0.124800,-0.072100,-0.126149,-0.107639,0.205908,-0.095745,-0.019618,-0.040055,-0.170137,-0.051207,-0.153658,0.181235,-0.134917,0.254167,-0.117261,-0.237504,0.123073,-0.179085,-0.169309,0.163762,0.173439,-0.049783,-0.185545,-0.022723,-0.037161,-0.120263,-0.060259,-0.130511,-0.047600,-0.096649,-0.152313
SUB_ID,0.996802,0.996802,1.000000,0.996802,1.000000,0.053670,0.056924,0.180474,0.017500,-0.071095,-0.008870,-0.006089,0.026679,-0.037152,-0.030402,-0.059036,-0.042714,-0.078726,-0.095660,0.048947,-0.019705,0.071645,0.059460,-0.045728,0.116648,0.011459,0.096052,0.125661,-0.195366,-0.054225,0.086588,0.097897,0.069067,0.101796,0.094620,-0.131374,0.223997,-0.054655,0.734246,0.519395,...,0.532840,0.642177,0.636457,0.579289,0.597081,-0.144298,-0.121505,0.036496,-0.115563,-0.124800,-0.072100,-0.126149,-0.107639,0.205908,-0.095745,-0.019618,-0.040055,-0.170137,-0.051207,-0.153658,0.191990,-0.135887,0.259552,-0.144752,-0.244675,0.128319,-0.197718,-0.157199,0.166623,0.160089,-0.037783,-0.194439,-0.038690,-0.034133,-0.118905,-0.068008,-0.139321,-0.055767,-0.137094,-0.158667
X,1.000000,1.000000,0.996802,1.000000,0.996802,0.061703,0.047760,0.181828,0.014324,-0.054216,-0.018130,-0.014932,0.016984,-0.031584,-0.027844,-0.055876,-0.039102,-0.062540,-0.105054,0.046482,-0.019601,0.067449,0.045814,-0.038527,0.121982,0.009640,0.099288,0.128881,-0.188254,-0.072971,0.138917,0.147769,0.128280,0.159167,0.156117,-0.258679,0.241644,-0.052009,0.748119,0.515331,...,0.527677,0.637955,0.630992,0.574395,0.591557,-0.144162,-0.121505,0.036496,-0.115563,-0.124800,-0.072100,-0.1

In [99]:
#drop the unnamed coulmns
selection.drop(selection.columns[selection.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

#corelation between the selected data
correlation_s = selection.corr()
correlation_s

,SUB_ID,X,subject,DX_GROUP,DSM_IV_TR,AGE_AT_SCAN,SEX,HANDEDNESS_SCORES,FIQ,VIQ,PIQ,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,ADI_R_RSRCH_RELIABLE,ADOS_MODULE,ADOS_TOTAL,ADOS_COMM,ADOS_SOCIAL,ADOS_STEREO_BEHAV,ADOS_RSRCH_RELIABLE,ADOS_GOTHAM_SOCAFFECT,ADOS_GOTHAM_RRB,ADOS_GOTHAM_TOTAL,ADOS_GOTHAM_SEVERITY,SRS_VERSION,SRS_RAW_TOTAL,SRS_AWARENESS,SRS_COGNITION,SRS_COMMUNICATION,SRS_MOTIVATION,SRS_MANNERISMS,SCQ_TOTAL,AQ_TOTAL,CURRENT_MED_STATUS,OFF_STIMULANTS_AT_SCAN,VINELAND_RECEPTIVE_V_SCALED,VINELAND_EXPRESSIVE_V_SCALED,VINELAND_WRITTEN_V_SCALED,...,VINELAND_PLAY_V_SCALED,VINELAND_COPING_V_SCALED,VINELAND_SOCIAL_STANDARD,VINELAND_SUM_SCORES,VINELAND_ABC_STANDARD,VINELAND_INFORMANT,WISC_IV_VCI,WISC_IV_PRI,WISC_IV_WMI,WISC_IV_PSI,WISC_IV_SIM_SCALED,WISC_IV_VOCAB_SCALED,WISC_IV_INFO_SCALED,WISC_IV_BLK_DSN_SCALED,WISC_IV_PIC_CON_SCALED,WISC_IV_MATRIX_SCALED,WISC_IV_DIGIT_SPAN_SCALED,WISC_IV_LET_NUM_SCALED,WISC_IV_CODING_SCALED,WISC_IV_SYM_SCALED,EYE_STATUS_AT_SCAN,AGE_AT_MPRAGE,BMI,anat_cnr,anat_efc,anat_fber,anat_fwhm,anat_qi1,anat_snr,func_efc,func_fber,func_fwhm,func_dvars,func_outlier,func_quality,func_mean_fd,func_num_fd,func_perc_fd,func_gsr,SUB_IN_SMP
SUB_ID,1.000000,0.996938,1.000000,0.061534,0.005388,0.017788,0.041123,-0.107386,-0.019436,0.008824,0.010219,0.008794,0.014432,-0.045434,0.016150,0.056463,-0.147022,0.089541,0.019450,0.113110,0.046731,0.112174,0.147594,0.026776,0.124033,0.149724,-0.218425,-0.048111,0.238681,0.128181,0.323434,0.345196,0.230688,-0.733100,0.499556,-0.052422,0.830689,0.508347,0.636436,0.320192,...,0.521023,0.649602,0.633794,0.572617,0.590972,-0.153806,-0.152362,0.058235,-0.144000,-0.128108,-0.113697,-0.162745,-0.113980,0.215041,-0.092078,-0.003277,-0.054556,-0.198592,-0.037467,-0.185474,0.101219,-0.121469,0.030873,-0.074936,-0.232204,0.144431,-0.188087,-0.190360,0.165186,0.179996,-0.042194,-0.186669,-0.063495,0.000774,-0.107835,-0.103853,-0.187568,-0.114884,-0.170916,-0.092060
X,0.996938,1.000000,0.996938,0.071853,-0.004486,0.027769,0.036261,-0.089955,-0.028026,0.003239,0.002724,0.010749,0.013005,-0.043626,0.014929,0.064725,-0.152517,0.080590,0.013095,0.102982,0.027596,0.109021,0.145718,0.017836,0.119289,0.146261,-0.212479,-0.067111,0.235207,0.123985,0.323751,0.340992,0.225573,-0.728229,0.513151,-0.051953,0.848588,0.504225,0.632527,0.315737,...,0.515889,0.644997,0.628199,0.567543,0.585247,-0.153647,-0.152362,0.058235,-0.144000,-0.128108,-0.113697,-0.162745,-0.113980,0.215041,-0.092078,-0.003277,-0.054556,-0.198592,-0.037467,-0.185474,0.089991,-0.120941,0.036560,-0.050977,-0.223516,0.136775,-0.169921,-0.201817,0.161516,0.193866,-0.056876,-0.176549,-0.045614,-0.004144,-0.109964,-0.093991,-0.174198,-0.101969,-0.126022,-0.089751
subject,1.000000,0.996938,1.000000,0.061534,0.005388,0.017788,0.041123,-0.107386,-0.019436,0.008824,0.010219,0.008794,0.014432,-0.045434,0.016150,0.056463,-0.147022,0.089541,0.019450,0.113110,0.046731,0.112174,0.147594,0.026776,0.124033,0.149724,-0.218425,-0.048111,0.238681,0.128181,0.323434,0.345196,0.230688,-0.733100,0.499556,-0.052422,0.830689,0.508347,0.636436,0.320192,...,0.521023,0.649602,0.633794,0.572617,0.590972,-0.153806,-0.152362,0.058235,-0.144000,-0.128108,-0.113697,-0.162745,-0.113980,0.215041,-0.092078,-0.003277,-0.054556,-0.198592,-0.037467,-0.185474,0.101219,-0.121469,0.030873,-0.074936,-0.232204,0.144431,-0.188087,-0.190360,0.165186,0.179996,-0.042194,-0.186669,-0.063495,0.000774,-0.107835,-0.103853,-0.187568,-0.114884,-0.170916,-0.092060
DX_GROUP,0.061534,0.071853,0.061534,1.000000,-0.811268,0.009793,0.071082,0.177723,0.147581,0.176438,0.045226,NaN,NaN,NaN,NaN,NaN,0.098919,-0.639783,-0.551117,-0.633477,-0.373408,NaN,-0.501635,-0.364098,-0.521204,-0.585461,-0.185306,-0.808617,-0.389744,-0.613455,-0.583561,-0.602387,-0.544225,-0.794246,-0.595627,-0.448767,-0.483046,0.628458,0.770905,0.483257,...,0.614030,0.784141,0.778149,0.742125,0.772192,-0.157542,0.386499,0.007469,0.424915,0.464502,0.296531,0.391092,0.356194,-0.163912,0.2

## Data preprocessing

In [100]:
from sklearn.impute import SimpleImputer

f_dataset = selection.loc[:,['SUB_ID','SEX','HANDEDNESS_CATEGORY','AGE_AT_SCAN','FIQ','VIQ','PIQ','ADOS_TOTAL',
                            'ADI_R_SOCIAL_TOTAL_A','ADI_R_VERBAL_TOTAL_BV','ADI_RRB_TOTAL_C','ADI_R_ONSET_TOTAL_D',
                            'SUB_IN_SMP','CURRENT_MED_STATUS','DSM_IV_TR']]

# The handedness field contains categorical data, since it has considerably large number of missing values, NaN's are replaced by string 'n/a' (not available) as a new category 
f_dataset.HANDEDNESS_CATEGORY = f_dataset.HANDEDNESS_CATEGORY.fillna('n/a')

# Replacing other missing values with mean of respective columns
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
f_dataset.iloc[:, 3:12] = imputer.fit_transform(f_dataset.iloc[:, 3:12])
        
# Dropping all rows which still contain any NaN values
f_dataset = f_dataset.dropna()
f_dataset.head(10)

,SUB_ID,SEX,HANDEDNESS_CATEGORY,AGE_AT_SCAN,FIQ,VIQ,PIQ,ADOS_TOTAL,ADI_R_SOCIAL_TOTAL_A,ADI_R_VERBAL_TOTAL_BV,ADI_RRB_TOTAL_C,ADI_R_ONSET_TOTAL_D,SUB_IN_SMP,CURRENT_MED_STATUS,DSM_IV_TR
0,50002,1,Ambi,16.77,103.0,116.0,89.0,12.000000,16.000000,9.000000,5.000000,4.000000,1,0.0,1.0
1,50003,1,R,24.45,124.0,128.0,115.0,13.000000,27.000000,22.000000,5.000000,3.000000,1,1.0,1.0
2,50004,1,R,19.09,113.0,108.0,117.0,18.000000,19.000000,12.000000,5.000000,3.000000,1,0.0,1.0
3,50005,2,R,13.73,119.0,117.0,118.0,12.000000,23.000000,19.000000,3.000000,4.000000,0,1.0,1.0
4,50006,1,L,13.37,109.0,99.0,119.0,12.000000,13.000000,10.000000,4.000000,3.000000,1,0.0,1.0
5,50007,1,R,17.78,110.0,106.0,112.0,17.000000,21.000000,14.000000,9.000000,1.000000,1,0.0,1.0
6,50008,1,R,32.45,123.0,123.0,114.0,16.000000,24.000000,20.000000,10.000000,2.000000,1,1.0,1.0
7,50009,1,R,33.86,126.0,118.0,128.0,10.000000,20.000000,11.000000,3.000000,2.000000,1,1.0,1.0
8,50010,1,L,35.20,81.0,81.0,93.0,10.876344,19.705882,15.739003,6.094118,3.242424,1,0.0,1.0
9,50011,1,L,16.93,111.0,101.0,120.0,13.000000,24.000000,14.000000,6.000000,1.000000,0,0.0,1.0


In [101]:
print(f_dataset['DSM_IV_TR'].value_counts())

# Features in X and resultant classes in y
X = np.array(f_dataset.iloc[:,1:14].values)
y = np.array(f_dataset.iloc[:,14].values)

0.0    403
1.0    231
2.0     71
3.0     14
Name: DSM_IV_TR, dtype: int64


In [102]:

# One-hot-encoding categorical data of HANDEDNESS_CATEGORY
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = columnTransformer.fit_transform(X)

# Feature Scaling
sc = StandardScaler()
X = sc.fit_transform(X)

# Splitting the dataset into the Training set and Test set in ratio 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [103]:
pip install -U scikit-learn

In [104]:
#  Dropping all rows which still contain any NaN values
f_dataset = f_dataset.dropna()

## Model Evaluation and predicting classes

In [105]:
model_list = ['Random Forest', 'Logistic Regression', 'kNN', 'SVM_rbf', 'ANN', 'XGBoost']
accuracies = [0]*6
auc_roc = [0]*6

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

In [106]:
from sklearn.ensemble import RandomForestClassifier

# Using the best parameters combination after tuning hyper-parameters
rf_classifier = RandomForestClassifier(n_estimators = 45, min_samples_split = 3, max_depth = 15, 
                                       bootstrap = False, random_state = 42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

accuracies[0] = accuracy_score(y_test, y_pred)
auc_roc[0] = multiclass_roc_auc_score(y_test, y_pred)

In [107]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver = 'saga', multi_class = 'ovr', max_iter = 500)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracies[1] = accuracy_score(y_test, y_pred)
auc_roc[1] = multiclass_roc_auc_score(y_test, y_pred)

In [108]:
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 5, weights = 'distance')
knn.fit(X_train, y_train) 
y_pred = knn.predict(X_test)  

accuracies[2] = accuracy_score(y_test, y_pred)
auc_roc[2] = multiclass_roc_auc_score(y_test, y_pred)

In [109]:
from sklearn.svm import SVC 
svm_model = SVC(kernel = 'rbf', gamma = 'auto', C=0.8)
svm_model.fit(X_train, y_train) 
y_pred = svm_model.predict(X_test) 

accuracies[3] = accuracy_score(y_test, y_pred)
auc_roc[3] = multiclass_roc_auc_score(y_test, y_pred)

In [116]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

def build_classifier():
    classifier = Sequential()
    # Adding the input layer and the first hidden layer 
    classifier.add(Dense(units = 15, activation = 'relu', input_dim = 17))
    # Adding a hidden layer
    classifier.add(Dense(units = 7, activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    # Adding a hidden layer
    classifier.add(Dense(units = 9, activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    # Adding a hidden layer
    classifier.add(Dense(units = 7, activation = 'relu'))
    classifier.add(Dropout(rate = 0.1))
    # Adding the output layer
    classifier.add(Dense(units = 4, activation = 'softmax'))
    # Compiling the ANN
    classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier

# Encoding the classes in y
y_binary = y

cv = KFold(n_splits=10, random_state=42, shuffle=True)
classifier = KerasClassifier(build_fn = build_classifier, batch_size=25, epochs=200)
acc_cv = cross_val_score(estimator = classifier, X = X, y = y_binary, cv = cv)

accuracies[4] = acc_cv.mean()
auc_roc[4] = ' '

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


Epoch 1/200
26/26 [==============================] - 1s 3ms/step - loss: 1.3638 - accuracy: 0.3570
Epoch 2/200
26/26 [==============================] - 0s 3ms/step - loss: 1.3165 - accuracy: 0.4513
Epoch 3/200
26/26 [==============================] - 0s 3ms/step - loss: 1.2847 - accuracy: 0.4946
Epoch 4/200
26/26 [==============================] - 0s 3ms/step - loss: 1.2305 - accuracy: 0.5178
Epoch 5/200
26/26 [==============================] - 0s 3ms/step - loss: 1.1746 - accuracy: 0.5641
Epoch 6/200
26/26 [==============================] - 0s 3ms/step - loss: 1.1103 - accuracy: 0.6167
Epoch 7/200
26/26 [==============================] - 0s 3ms/step - loss: 1.0331 - accuracy: 0.6770
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 0.9468 - accuracy: 0.6847
Epoch 9/200
26/26 [==============================] - 0s 3ms/step - loss: 0.8808 - accuracy: 0.7110
Epoch 10/200
26/26 [==============================] - 0s 2ms/step - loss: 0.8843 - accuracy: 0.6878
Epoch 11/

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 105, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 313, in score
    outputs = self.model.evaluate(x, y, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  Fil

Epoch 1/200
26/26 [==============================] - 1s 3ms/step - loss: 1.3637 - accuracy: 0.4730
Epoch 2/200
26/26 [==============================] - 0s 2ms/step - loss: 1.2941 - accuracy: 0.5765
Epoch 3/200
26/26 [==============================] - 0s 2ms/step - loss: 1.2370 - accuracy: 0.5611
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 1.1863 - accuracy: 0.5889
Epoch 5/200
26/26 [==============================] - 0s 2ms/step - loss: 1.1305 - accuracy: 0.5966
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 1.0688 - accuracy: 0.6244
Epoch 7/200
26/26 [==============================] - 0s 2ms/step - loss: 1.0605 - accuracy: 0.6043
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 0.9814 - accuracy: 0.6754
Epoch 9/200
26/26 [==============================] - 0s 3ms/step - loss: 0.9518 - accuracy: 0.6955
Epoch 10/200
26/26 [==============================] - 0s 2ms/step - loss: 0.9297 - accuracy: 0.6692
Epoch 11/

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 105, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 418, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/wrappers/scikit_learn.py", line 313, in score
    outputs = self.model.evaluate(x, y, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/traceback_utils.py", line 67, in error_handler
    raise e.with_traceback(filtered_tb) from None
  Fil

Epoch 1/200
26/26 [==============================] - 1s 2ms/step - loss: 1.3678 - accuracy: 0.3688
Epoch 2/200
26/26 [==============================] - 0s 2ms/step - loss: 1.3256 - accuracy: 0.4985
Epoch 3/200
26/26 [==============================] - 0s 2ms/step - loss: 1.2808 - accuracy: 0.5309
Epoch 4/200
26/26 [==============================] - 0s 2ms/step - loss: 1.2257 - accuracy: 0.5525
Epoch 5/200
26/26 [==============================] - 0s 2ms/step - loss: 1.1723 - accuracy: 0.5448
Epoch 6/200
26/26 [==============================] - 0s 2ms/step - loss: 1.1152 - accuracy: 0.5849
Epoch 7/200
26/26 [==============================] - 0s 3ms/step - loss: 1.0830 - accuracy: 0.5910
Epoch 8/200
26/26 [==============================] - 0s 2ms/step - loss: 1.0545 - accuracy: 0.5895
Epoch 9/200
26/26 [==============================] - 0s 3ms/step - loss: 1.0383 - accuracy: 0.5957
Epoch 10/200
26/26 [==============================] - 0s 3ms/step - loss: 1.0311 - accuracy: 0.5926
Epoch 11/

In [117]:
from xgboost import XGBClassifier

# Create a classifier using best params
xgb = XGBClassifier(booster='gbtree', objective='multi:softmax', max_depth = 11, n_estimators = 35,
                    eta = 0.1, subsample  = 0.7, num_class = 4, random_state=42)
# Fit the classifier with the training data
xgb.fit(X_train,y_train)
# Use trained model to predict output of test dataset
y_pred = xgb.predict(X_test)

accuracies[5] = accuracy_score(y_test, y_pred)
auc_roc[5] = multiclass_roc_auc_score(y_test, y_pred)

print('CLASSIFICATION REPORT for XGBoost classifier... \n')
print(classification_report(y_test,y_pred))
print('\nCONFUSION MATRIX for XGBoost classifier... \n')
confusion_matrix(y_test,y_pred)

CLASSIFICATION REPORT for XGBoost classifier... 

              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98        81
         1.0       0.75      0.93      0.83        45
         2.0       0.60      0.19      0.29        16
         3.0       0.00      0.00      0.00         2

    accuracy                           0.87       144
   macro avg       0.58      0.53      0.52       144
weighted avg       0.84      0.87      0.84       144


CONFUSION MATRIX for XGBoost classifier... 



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[80,  1,  0,  0],
       [ 1, 42,  2,  0],
       [ 2, 11,  3,  0],
       [ 0,  2,  0,  0]])

In [118]:
models = {'Classifiers': model_list, 'Accuracy score': accuracies, 'AUC-ROC Score': auc_roc}
models_df = pd.DataFrame(models, columns = ['Classifiers', 'Accuracy score', 'AUC-ROC Score'] )
models_df

,Classifiers,Accuracy score,AUC-ROC Score
0,Random Forest,0.854167,0.724428
1,Logistic Regression,0.673611,0.592203
2,kNN,0.743056,0.656717
3,SVM_rbf,0.833333,0.700449
4,ANN,NaN,
5,XGBoost,0.868056,0.737979
